In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import gc
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
import warnings
from datetime import date, timedelta
warnings.filterwarnings('ignore')

In [2]:
sku_basic_info=pd.read_csv('data/jdata_sku_basic_info.csv')
user_basic_info=pd.read_csv('data/jdata_user_basic_info.csv')
user_action=pd.read_csv('data/jdata_user_action.csv')
user_order=pd.read_csv('data/jdata_user_order.csv')
user_comment_score=pd.read_csv('data/jdata_user_comment_score.csv')
print sku_basic_info.shape,user_basic_info.shape,user_action.shape,user_order.shape,user_comment_score.shape



(108333, 6) (99446, 4) (8033923, 5) (881692, 6) (233366, 4)


In [3]:
%run com_util.py

In [13]:
#target_month=201705
#start_month=201704
#feature_month=[start_month,target_month]

def get_order_month_feature(feature_month,name):
    print 'o_start'
    user_order=pd.read_csv('data/jdata_user_order.csv')
    user_order = user_order.merge(sku_basic_info,on=['sku_id'],how='left')
    user_order = user_order.merge(user_basic_info,on=['user_id'],how='left')
    user_order = user_order.merge(user_comment_score,on=['user_id','o_id'],how='left')
    # user_order['y_m_date'] = 100*pd.to_datetime(user_order['o_date']).dt.year+pd.to_datetime(user_order['o_date']).dt.month

    FLAG='O_'+str(name)+'_'
    temp_user_order=user_order[(user_order.o_date<=feature_month[1])&(user_order.o_date>=feature_month[0])]
    temp_user_order['comment_gap']=(pd.to_datetime(temp_user_order.comment_create_tm)-pd.to_datetime(temp_user_order.o_date)).dt.days.fillna(0)
    temp_user_order['o_date'] = pd.to_datetime(temp_user_order['o_date'])
    temp_user_order['order_day'] = temp_user_order['o_date'].dt.day
    temp_user_order['comment_create_tm'] = pd.to_datetime(temp_user_order['comment_create_tm'])
    temp_user_order['comment_day'] = temp_user_order['comment_create_tm'].dt.day
    temp_user_order['once_cost'] = temp_user_order.price*temp_user_order.o_sku_num
    #  
    sample = feat_count(user_basic_info[['user_id']],temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'sku_id',FLAG+'30101_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'cate',FLAG+'no_30101_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'cate',FLAG+'30_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'cate',FLAG+'101_')

    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'o_id',FLAG+'30_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'o_id',FLAG+'101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_id',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_id',FLAG+'no_30101_')

    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'once_cost',FLAG+'30_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'once_cost',FLAG+'101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'once_cost',FLAG+'30101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'once_cost',FLAG+'no30101_')
    
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'o_sku_num',FLAG+'30_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'o_sku_num',FLAG+'101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_sku_num',FLAG+'30101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_sku_num',FLAG+'no30101_')

    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no30_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==101))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no101_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate!=101)&(temp_user_order.cate!=30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no30101_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==101)|(temp_user_order.cate==30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'comment_create_tm',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'comment_create_tm',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'comment_create_tm',FLAG+'30_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'comment_create_tm',FLAG+'101_')
  #/////////////   mei you zhe xie xia jiang le a ? ?
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'comment_day',FLAG+'no301010_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'comment_day',FLAG+'30101_')

    sample = feat_min(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'order_day',FLAG+'no301010_')
    sample = feat_min(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30101_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'order_day',FLAG+'no301010_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30101_')
 #///////////////////you sha yong ? ?
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_date',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_date',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'o_date',FLAG+'101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'o_date',FLAG+'30_')
    temp_user_order_0 = temp_user_order.copy()
    temp_user_order_0 = merge_count(temp_user_order_0,['user_id'],'o_id','order_count')
    temp_user_order_0 = merge_max(temp_user_order_0,['user_id'],'o_date','order_max')
    temp_user_order_0 = merge_min(temp_user_order_0,['user_id'],'o_date','order_min')
    temp_user_order_0[FLAG+'feature_u1'] = (temp_user_order_0.order_max-temp_user_order_0.order_min).dt.days/(temp_user_order_0.order_count)
    temp_user_order_0 = temp_user_order_0[['user_id',FLAG+'feature_u1']]
    temp_user_order_0 = temp_user_order_0.drop_duplicates()
    sample = sample.merge(temp_user_order_0,on=['user_id'],how='left')
    sample=sample.drop(['user_id'],axis=1)
    print user_order.shape,temp_user_order.shape,sample.shape
    gc.collect()

    return sample,sample.columns.tolist()


In [14]:
def get_order_one_month_feature(feature_month,name):
    print 'o_start'
    user_order=pd.read_csv('data/jdata_user_order.csv')
    user_order = user_order.merge(sku_basic_info,on=['sku_id'],how='left')
    user_order = user_order.merge(user_basic_info,on=['user_id'],how='left')
    user_order = user_order.merge(user_comment_score,on=['user_id','o_id'],how='left')
    # user_order['y_m_date'] = 100*pd.to_datetime(user_order['o_date']).dt.year+pd.to_datetime(user_order['o_date']).dt.month

    FLAG='O_'+str(name)+'_'
    temp_user_order=user_order[(user_order.o_date<=feature_month[1])&(user_order.o_date>=feature_month[0])]
    temp_user_order['comment_gap']=(pd.to_datetime(temp_user_order.comment_create_tm)-pd.to_datetime(temp_user_order.o_date)).dt.days.fillna(0)
    temp_user_order['o_date'] = pd.to_datetime(temp_user_order['o_date'])
    temp_user_order['order_day'] = temp_user_order['o_date'].dt.day    
    temp_user_order['comment_create_tm'] = pd.to_datetime(temp_user_order['comment_create_tm'])
    temp_user_order['comment_day'] = temp_user_order['comment_create_tm'].dt.day
    temp_user_order['once_cost'] = temp_user_order.price*temp_user_order.o_sku_num
    #  
    sample = feat_count(user_basic_info[['user_id']],temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'sku_id',FLAG+'30101_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'cate',FLAG+'no_30101_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'cate',FLAG+'30_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'cate',FLAG+'101_')

    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'o_id',FLAG+'30_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'o_id',FLAG+'101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_id',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_id',FLAG+'no_30101_')

    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'once_cost',FLAG+'30_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'once_cost',FLAG+'101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'once_cost',FLAG+'30101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'once_cost',FLAG+'no30101_')
    
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'o_sku_num',FLAG+'30_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'o_sku_num',FLAG+'101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_sku_num',FLAG+'30101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_sku_num',FLAG+'no30101_')

    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no30_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==101))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no101_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate!=101)&(temp_user_order.cate!=30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no30101_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==101)|(temp_user_order.cate==30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_30101_')
    sample = feat_min(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'comment_day',FLAG+'no301010_')
    sample = feat_min(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'comment_day',FLAG+'30101_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'comment_day',FLAG+'no301010_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'comment_day',FLAG+'30101_')
  
    
    sample = feat_min(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'order_day',FLAG+'no301010_')
    sample = feat_min(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30101_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'order_day',FLAG+'no301010_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'order_day',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'order_day',FLAG+'101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30_')
    temp_user_order_0 = temp_user_order.copy()
    temp_user_order_0 = merge_count(temp_user_order_0,['user_id'],'o_id','order_count')
    temp_user_order_0 = merge_max(temp_user_order_0,['user_id'],'o_date','order_max')
    temp_user_order_0 = merge_min(temp_user_order_0,['user_id'],'o_date','order_min')
    temp_user_order_0[FLAG+'feature_u1'] = (temp_user_order_0.order_max-temp_user_order_0.order_min).dt.days/(temp_user_order_0.order_count)
    temp_user_order_0 = temp_user_order_0[['user_id',FLAG+'feature_u1']]
    temp_user_order_0 = temp_user_order_0.drop_duplicates()
    sample = sample.merge(temp_user_order_0,on=['user_id'],how='left')
    sample[FLAG+'gap_end'] = pd.to_datetime(feature_month[1]).day-sample[FLAG+'30101_order_day_user_id_max']
    sample[FLAG+'gap_end_no'] = pd.to_datetime(feature_month[1]).day-sample[FLAG+'no301010_order_day_user_id_max']
    sample=sample.drop(['user_id'],axis=1)
    print user_order.shape,temp_user_order.shape,sample.shape
    gc.collect()

    return sample,sample.columns.tolist()

In [15]:
#feature 逐个月份是否购买　购买数目

# 聚焦S1
def get_order_every_month_feature(feature_month,name):
    print 'o_start'
    user_order=pd.read_csv('data/jdata_user_order.csv')
    user_order = user_order.merge(sku_basic_info,on=['sku_id'],how='left')
    user_order = user_order.merge(user_basic_info,on=['user_id'],how='left')
    user_order = user_order.merge(user_comment_score,on=['user_id','o_id'],how='left')
    # user_order['y_m_date'] = 100*pd.to_datetime(user_order['o_date']).dt.year+pd.to_datetime(user_order['o_date']).dt.month

    FLAG='Every_O_'+str(name)+'_'
    temp_user_order=user_order[(user_order.o_date<=feature_month[1])&(user_order.o_date>=feature_month[0])]
    temp_user_order['comment_gap']=(pd.to_datetime(temp_user_order.comment_create_tm)-pd.to_datetime(temp_user_order.o_date)).dt.days.fillna(0)
    temp_user_order['o_date'] = pd.to_datetime(temp_user_order['o_date'])
    temp_user_order['order_day'] = temp_user_order['o_date'].dt.day
    temp_user_order['comment_create_tm'] = pd.to_datetime(temp_user_order['comment_create_tm'])
    temp_user_order['comment_day'] = temp_user_order['comment_create_tm'].dt.day
    #  
    sample = feat_count(user_basic_info[['user_id']],temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'sku_id',FLAG+'30101_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'cate',FLAG+'no_30101_')

    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_id',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_id',FLAG+'no_30101_')

    sample = feat_mean(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'price',FLAG+'30101_')
    sample = feat_mean(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'price',FLAG+'no30101_')

    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_sku_num',FLAG+'30101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_sku_num',FLAG+'no30101_')
    
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'comment_create_tm',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'comment_create_tm',FLAG+'30101_')
     
    
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_date',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_date',FLAG+'30101_')
    sample=sample.drop(['user_id'],axis=1)
    print user_order.shape,temp_user_order.shape,sample.shape
    gc.collect()

    return sample,sample.columns.tolist()

In [16]:
#target_month=201705
#start_month=201704
#feature_month=[start_month,target_month]
def get_action_month_feature(feature_month,name):
    print 'action_start'
    FLAG='A_'+str(name)+'_'
    user_action=pd.read_csv('data/jdata_user_action.csv')
    user_action = user_action.merge(sku_basic_info,on=['sku_id'],how='left')
    user_action = user_action.merge(user_basic_info,on=['user_id'],how='left')

    #user_action['y_m_date'] = 100*pd.to_datetime(user_action['a_date']).dt.year+pd.to_datetime(user_action['a_date']).dt.month
    temp_user_action=user_action[(user_action.a_date<=feature_month[1])&(user_action.a_date>=feature_month[0])]


    sample = feat_nunique(user_basic_info[['user_id']],temp_user_action,['user_id'],'sku_id',FLAG)
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'sku_id',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)],['user_id'],'sku_id',FLAG+'101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==30)],['user_id'],'sku_id',FLAG+'30_')

    sample = feat_nunique(sample,temp_user_action[temp_user_action.a_type==2],['user_id'],'sku_id',FLAG+'type2_')
    sample = feat_nunique(sample,temp_user_action[temp_user_action.a_type==1],['user_id'],'sku_id',FLAG+'type1_')
    sample = feat_sum(sample,temp_user_action[temp_user_action.a_type==1],['user_id'],'a_num',FLAG+'type1_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type2_30_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type1_30')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==30))],['user_id'],'a_num',FLAG+'type1_30')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate==101))],['user_id'],'sku_id',FLAG+'type2_101_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101))],['user_id'],'sku_id',FLAG+'type1_101')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101))],['user_id'],'a_num',FLAG+'type1_101')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type2_30101_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type1_30101')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'a_num',FLAG+'type1_30101')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'sku_id',FLAG+'type2_no30101_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'sku_id',FLAG+'type1_no30101')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'a_num',FLAG+'type1_no30101')

    
    
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate!=101)&(temp_user_action.cate!=30)],['user_id'],'a_date',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'a_date',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==30)],['user_id'],'a_date',FLAG+'30_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)],['user_id'],'a_date',FLAG+'101_')
    sample=sample.drop(['user_id'],axis=1)
    print user_action.shape,temp_user_action.shape,sample.shape
    gc.collect()

    return sample,sample.columns.tolist()


In [17]:
def get_action_one_month_feature(feature_month,name):
    print 'action_start'
    FLAG='A_'+str(name)+'_'
    user_action=pd.read_csv('data/jdata_user_action.csv')
    user_action = user_action.merge(sku_basic_info,on=['sku_id'],how='left')
    user_action = user_action.merge(user_basic_info,on=['user_id'],how='left')

    #user_action['y_m_date'] = 100*pd.to_datetime(user_action['a_date']).dt.year+pd.to_datetime(user_action['a_date']).dt.month
    temp_user_action=user_action[(user_action.a_date<=feature_month[1])&(user_action.a_date>=feature_month[0])]
    temp_user_action['action_day'] = pd.to_datetime(temp_user_action['a_date']).dt.day
    temp_user_action['action_month'] = pd.to_datetime(temp_user_action['a_date']).dt.month

    sample = feat_nunique(user_basic_info[['user_id']],temp_user_action,['user_id'],'sku_id',FLAG)
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'sku_id',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)],['user_id'],'sku_id',FLAG+'101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==30)],['user_id'],'sku_id',FLAG+'30_')

    sample = feat_nunique(sample,temp_user_action[temp_user_action.a_type==2],['user_id'],'sku_id',FLAG+'type2_')
    sample = feat_nunique(sample,temp_user_action[temp_user_action.a_type==1],['user_id'],'sku_id',FLAG+'type1_')
    sample = feat_sum(sample,temp_user_action[temp_user_action.a_type==1],['user_id'],'a_num',FLAG+'type1_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type2_30_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type1_30')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==30))],['user_id'],'a_num',FLAG+'type1_30')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate==101))],['user_id'],'sku_id',FLAG+'type2_101_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101))],['user_id'],'sku_id',FLAG+'type1_101')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101))],['user_id'],'a_num',FLAG+'type1_101')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type2_30101_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type1_30101')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'a_num',FLAG+'type1_30101')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'sku_id',FLAG+'type2_no30101_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'sku_id',FLAG+'type1_no30101')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'a_num',FLAG+'type1_no30101')

    
    sample = feat_min(sample,temp_user_action[(temp_user_action.cate!=101)&(temp_user_action.cate!=30)],['user_id'],'action_day',FLAG+'no301010_')
    sample = feat_min(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'action_day',FLAG+'30101_')
    sample = feat_max(sample,temp_user_action[(temp_user_action.cate!=101)&(temp_user_action.cate!=30)],['user_id'],'action_day',FLAG+'no301010_')
    sample = feat_max(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'action_day',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate!=101)&(temp_user_action.cate!=30)],['user_id'],'action_day',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'action_day',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)],['user_id'],'action_day',FLAG+'101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==30)],['user_id'],'action_day',FLAG+'30_')
    sample[FLAG+'gap_end'] = pd.to_datetime(feature_month[1]).day-sample[FLAG+'30101_action_day_user_id_max']
    sample=sample.drop(['user_id'],axis=1)
    print user_action.shape,temp_user_action.shape,sample.shape
    gc.collect()

    return sample,sample.columns.tolist()

In [18]:
#feature_month=[start , end]
def get_action_order_feature(feature_month):
    print 'a_o_start'
    user_action=pd.read_csv('data/jdata_user_action.csv')
    user_order=pd.read_csv('data/jdata_user_order.csv')
    user_action_order = pd.merge(user_action,user_order,on=['user_id','sku_id'],how='left')
    user_action_order = user_action_order.merge(sku_basic_info,on=['sku_id'],how='left')
    print user_action_order.shape
    user_action_order = user_action_order[((pd.to_datetime(user_action_order.a_date)<=pd.to_datetime(user_action_order.o_date.fillna('2018-01-01'))))]
    user_action_order = user_action_order.sort_values(by=['user_id','sku_id','a_date','o_date']).drop_duplicates(['user_id','sku_id','a_date'],keep="first")
    user_action_order = user_action_order[['user_id','sku_id','a_date','o_date','cate']]
    user_action_order['o_a_gap']=(pd.to_datetime(user_action_order.o_date)-pd.to_datetime(user_action_order.a_date)).dt.days 
    FLAG='a_o_'
    print user_action_order.shape
    user_action_order = user_action_order[((user_action_order.a_date>=feature_month[0])|(user_action_order.a_date<feature_month[1]))&((user_action_order.o_date>=feature_month[0])|(user_action_order.o_date<feature_month[1]))]
    print user_action_order.shape
    sample = feat_count(user_basic_info[['user_id']],user_action_order[user_action_order.o_a_gap>=-1],['user_id'],'sku_id',FLAG+'order_')
    sample = feat_count(sample,user_action_order,['user_id'],'sku_id',FLAG)
    sample['a_o_sku_id_rate']=(sample.a_o_order_sku_id_user_id_count+1)/(sample.a_o_sku_id_user_id_count+1)
    sample = feat_mean(sample,user_action_order[user_action_order.o_a_gap>=-1],['user_id'],'o_a_gap',FLAG)
    sample = feat_mean(sample,user_action_order[(user_action_order.o_a_gap>=-1)&((user_action_order.cate==101)|(user_action_order.cate==30))],['user_id'],'o_a_gap',FLAG+'30101_')

    sample=sample.drop(['user_id'],axis=1)
    print sample.shape
    gc.collect()

    return sample,sample.columns.tolist()

In [19]:
# train / val label
#target_month=201704

def get_label(target_month,month):
    #user_action=pd.read_csv('data/jdata_user_action.csv')
    user_order=pd.read_csv('data/jdata_user_order.csv')
    sku_basic_info=pd.read_csv('data/jdata_sku_basic_info.csv')
    user_basic_info=pd.read_csv('data/jdata_user_basic_info.csv')

    user_order = user_order.merge(sku_basic_info,on='sku_id',how='left')
    #user_action['y_m_date'] = 100*pd.to_datetime(user_action['a_date']).dt.year+pd.to_datetime(user_action['a_date']).dt.month
    #user_order['y_m_date'] = 100*pd.to_datetime(user_order['o_date']).dt.year+pd.to_datetime(user_order['o_date']).dt.month
    #user_action = user_action[['user_id','a_date']]
    user_order = user_order[['user_id','o_date','cate','o_id']]
    #user_action = user_action[user_action.y_m_date==(target_month-1)]
    #user_order = user_order[user_order.y_m_date==target_month]
    user_order = user_order[(user_order.o_date>=target_month[0])&(user_order.o_date<=target_month[1])]
    user_order = user_order[(user_order.cate==30)|(user_order.cate==101)]
    print user_order.shape
    
    sample = feat_nunique(user_basic_info[['user_id']],user_order,['user_id'],'o_id')
    sample.columns=['user_id','label_1']
    user_order = user_order.sort_values(by=['user_id','o_date']).drop_duplicates(["user_id"], keep="first")
   
    #user_action = user_action.sort_values(by=['user_id','a_date']).drop_duplicates(["user_id"], keep="last")
    label = pd.merge(user_basic_info[['user_id']],user_order[['user_id','o_date']],on=['user_id'],how='left')
    #label = label.merge(user_action[['user_id','a_date']],on=['user_id'],how='left')
    label = label.merge(sample,on=['user_id'],how='left')
    label['label_2'] = (pd.to_datetime(label["o_date"])-pd.to_datetime(date(2017,month,1))).dt.days.fillna(0)
    print label.shape
    return label


In [20]:
#train feature 201703
user_basic_info=pd.read_csv('data/jdata_user_basic_info.csv')

train_order_1,_ = get_order_one_month_feature(['2017-07-01','2017-07-31'],1)
train_order_2,_ = get_order_month_feature(['2017-05-01','2017-07-31'],2) 
train_order_3,_ = get_order_month_feature(['2017-02-01','2017-07-31'],3) 
train_order_4,_ = get_order_month_feature(['2016-11-01','2017-07-31'],4) 

train_action_1,_ = get_action_one_month_feature(['2017-07-01','2017-07-31'],1)
train_action_2,_ = get_action_month_feature(['2017-05-01','2017-07-31'],2) 
train_action_3,_ = get_action_month_feature(['2017-02-01','2017-07-31'],3) 
train_action_4,_ = get_action_month_feature(['2016-11-01','2017-07-31'],4) 
#train_action_order_0,_ = get_action_order_feature(['2016-11-01','2017-03-31'])
train_feature = pd.concat([user_basic_info[['age','sex','user_lv_cd']],train_order_1,train_order_2,train_order_3,
                           train_action_1,train_action_2,train_action_3,train_order_4,train_action_4],axis=1)
print train_feature.shape
# train label
train_label = get_label(['2017-08-01','2017-08-30'],8)

train_feature_label1 = pd.concat([train_feature,train_label],axis=1)

 
#val feature 201704
val_order_1,_ = get_order_one_month_feature(['2017-06-01','2017-06-30'],1)
val_order_2,_ = get_order_month_feature(['2017-04-01','2017-06-30'],2)
val_order_3,_ = get_order_month_feature(['2017-01-01','2017-06-30'],3)
val_order_4,_ = get_order_month_feature(['2016-10-01','2017-06-30'],4)

#val_order_4,_ = get_order_every_month_feature(['2017-01-01','2017-01-31'],2)
#val_order_5,_ = get_order_every_month_feature(['2017-01-01','2017-02-28'],12)
#val_order_6,_ = get_order_every_month_feature(['2016-12-01','2016-12-30'],3)

val_action_1,_ = get_action_one_month_feature(['2017-06-01','2017-06-30'],1)
val_action_2,_ = get_action_month_feature(['2017-04-01','2017-06-30'],2)
val_action_3,_ = get_action_month_feature(['2017-01-01','2017-06-30'],3)
val_action_4,_ = get_action_month_feature(['2016-10-01','2017-06-30'],4)
#val_action_order_0,_ = get_action_order_feature(['2016-10-01','2017-02-28'])
val_feature = pd.concat([user_basic_info[['age','sex','user_lv_cd']],val_order_1,val_order_2,val_order_3,
                      #   val_order_5,
                         val_action_1,val_action_2,val_action_3,val_order_4,val_action_4],axis=1)
# val label
val_label = get_label(['2017-07-01','2017-07-31'],7)
print val_feature.shape

val_feature_label1 = pd.concat([val_feature,val_label],axis=1)
 
#test feature 201708
test_order_1,_ = get_order_one_month_feature(['2017-08-01','2017-08-31'],1)
test_order_2,_ = get_order_month_feature(['2017-06-01','2017-08-31'],2)
test_order_3,_ = get_order_month_feature(['2017-03-01','2017-08-31'],3)
test_order_4,_ = get_order_month_feature(['2016-12-01','2017-08-31'],4)

#test_order_4,_ = get_order_every_month_feature(['2017-03-01','2017-03-31'],2)
#test_order_5,_ = get_order_every_month_feature(['2017-03-01','2017-04-30'],12)
#test_order_6,_ = get_order_every_month_feature(['2017-02-01','2017-02-28'],3)

test_action_1,_ = get_action_one_month_feature(['2017-08-01','2017-08-31'],1)
test_action_2,_ = get_action_month_feature(['2017-06-01','2017-08-31'],2)
test_action_3,_ = get_action_month_feature(['2017-03-01','2017-08-31'],3)
test_action_4,_ = get_action_month_feature(['2016-12-01','2017-08-31'],4)
#test_action_order_0,_ = get_action_order_feature(['2016-12-01','2017-05-01'])
test_feature1 = pd.concat([user_basic_info[['age','sex','user_lv_cd']],test_order_1,test_order_2,test_order_3,
                        #   test_order_5,
                           test_action_1,test_action_3,test_action_2,test_order_4,test_action_4],axis=1)
print test_feature1.shape
 

o_start
(993571, 16) (101327, 20) (99446, 35)
o_start
(993571, 16) (364330, 20) (99446, 35)
o_start
(993571, 16) (598915, 20) (99446, 35)
o_start
(993571, 16) (789351, 20) (99446, 35)
action_start
(8033923, 13) (793544, 15) (99446, 28)
action_start
(8033923, 13) (2781988, 13) (99446, 23)
action_start
(8033923, 13) (4733748, 13) (99446, 23)
action_start
(8033923, 13) (6343178, 13) (99446, 23)
(99446, 240)
(63977, 4)
(99446, 4)
o_start
(993571, 16) (172033, 20) (99446, 35)
o_start
(993571, 16) (352082, 20) (99446, 35)
o_start
(993571, 16) (557825, 20) (99446, 35)
o_start
(993571, 16) (739871, 20) (99446, 35)
action_start
(8033923, 13) (1256200, 15) (99446, 28)
action_start
(8033923, 13) (2700968, 13) (99446, 23)
action_start
(8033923, 13) (4427963, 13) (99446, 23)
action_start
(8033923, 13) (6011650, 13) (99446, 23)
(60735, 4)
(99446, 4)
(99446, 240)
o_start
(993571, 16) (109204, 20) (99446, 35)
o_start
(993571, 16) (382564, 20) (99446, 35)
o_start
(993571, 16) (643509, 20) (99446, 35)
o

In [21]:
def label_change(data):
    if(data>=1):
        return data
    else:
        return 0
train_feature_label1['label_1'] = train_feature_label1.label_1.apply(label_change)
val_feature_label1['label_1'] = val_feature_label1.label_1.apply(label_change)

In [ ]:
# model predict

In [22]:
feature_list = test_feature1.columns.tolist()
# test_feature
print len(feature_list)

240


In [23]:
####S1
params_1 = {
          'boosting_type': 'gbdt',
          'objective': 'regression_l2',
          'metric': 'auc',
          'min_child_weight': 1.5,
          'num_leaves': 2**5,
          'lambda_l2': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.5,
          'colsample_bylevel': 0.5,
          'learning_rate': 0.1,
          'scale_pos_weight': 20,
          'seed': 2018,
          'bagging_fraction': 0.8,
          'bagging_freq': 5,
          'nthread': 4,
          'silent': True,
          }

In [36]:
from sklearn.model_selection import train_test_split
test_cv = test_feature1.copy()

for bn in range(3):
    X_train, X_test, y_train, y_test = train_test_split(train_feature_label1[feature_list], train_feature_label1['label_1'], test_size=0.3, random_state=bn)
    train_matrix=lgb.Dataset(X_train,label=y_train)
    valid_matrix=lgb.Dataset(X_test,label=y_test)
    model_A = lgb.train(params_1, train_matrix, num_boost_round=4000, valid_sets=valid_matrix,
                      early_stopping_rounds=100 )
    test_cv["prob"+str(bn)] = model_A.predict(test_cv[feature_list], num_iteration=model.best_iteration).reshape((test_cv.shape[0], 1))
test_cv = pd.concat([user_basic_info[['user_id']],test_cv[['prob0','prob1','prob2']]],axis=1)
test_cv['prob'] = (test_cv.prob0+test_cv.prob1+test_cv.prob2)/3
test_cv = test_cv.sort_values("prob",ascending=False)

[1]	valid_0's auc: 0.779366
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.787576
[3]	valid_0's auc: 0.792513
[4]	valid_0's auc: 0.793404
[5]	valid_0's auc: 0.794601
[6]	valid_0's auc: 0.794953
[7]	valid_0's auc: 0.795288
[8]	valid_0's auc: 0.795787
[9]	valid_0's auc: 0.795991
[10]	valid_0's auc: 0.796726
[11]	valid_0's auc: 0.797945
[12]	valid_0's auc: 0.798638
[13]	valid_0's auc: 0.798728
[14]	valid_0's auc: 0.799298
[15]	valid_0's auc: 0.799511
[16]	valid_0's auc: 0.799621
[17]	valid_0's auc: 0.80008
[18]	valid_0's auc: 0.80038
[19]	valid_0's auc: 0.800305
[20]	valid_0's auc: 0.800809
[21]	valid_0's auc: 0.801297
[22]	valid_0's auc: 0.801516
[23]	valid_0's auc: 0.801678
[24]	valid_0's auc: 0.801683
[25]	valid_0's auc: 0.801709
[26]	valid_0's auc: 0.8017
[27]	valid_0's auc: 0.801872
[28]	valid_0's auc: 0.802131
[29]	valid_0's auc: 0.802284
[30]	valid_0's auc: 0.802379
[31]	valid_0's auc: 0.802455
[32]	valid_0's auc: 0.802558
[33]	valid_0's auc: 0.

[130]	valid_0's auc: 0.808315
[131]	valid_0's auc: 0.808306
[132]	valid_0's auc: 0.808335
[133]	valid_0's auc: 0.808332
[134]	valid_0's auc: 0.808304
[135]	valid_0's auc: 0.808298
[136]	valid_0's auc: 0.808331
[137]	valid_0's auc: 0.808258
[138]	valid_0's auc: 0.80827
[139]	valid_0's auc: 0.80824
[140]	valid_0's auc: 0.808318
[141]	valid_0's auc: 0.808353
[142]	valid_0's auc: 0.808362
[143]	valid_0's auc: 0.80833
[144]	valid_0's auc: 0.808375
[145]	valid_0's auc: 0.808399
[146]	valid_0's auc: 0.808381
[147]	valid_0's auc: 0.808384
[148]	valid_0's auc: 0.808254
[149]	valid_0's auc: 0.808219
[150]	valid_0's auc: 0.808117
[151]	valid_0's auc: 0.80812
[152]	valid_0's auc: 0.808131
[153]	valid_0's auc: 0.808063
[154]	valid_0's auc: 0.808097
[155]	valid_0's auc: 0.808103
[156]	valid_0's auc: 0.80809
[157]	valid_0's auc: 0.808074
[158]	valid_0's auc: 0.808141
[159]	valid_0's auc: 0.808113
[160]	valid_0's auc: 0.808155
[161]	valid_0's auc: 0.80813
[162]	valid_0's auc: 0.808133
[163]	valid_0's 

In [25]:
####S2
params_2 = {
          'boosting_type': 'gbdt',
          'objective': 'regression_l2',
          'metric': 'mse',
          'min_child_weight': 1.5,
          'num_leaves': 2**5,
          'lambda_l2': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.5,
          'colsample_bylevel': 0.5,
          'learning_rate': 0.1,
          'scale_pos_weight': 20,
          'seed': 2018,
          'nthread': 4,
          'silent': True,
          }

In [26]:
##S2
train_x = train_feature_label1[train_feature_label1.label_1>=1][feature_list].values
train_y = train_feature_label1[train_feature_label1.label_1>=1]['label_2'].values

val_x = val_feature_label1[val_feature_label1.label_1>=1][feature_list].values
val_y = val_feature_label1[val_feature_label1.label_1>=1]['label_2'].values

train_matrix=lgb.Dataset(train_x,label=train_y)
valid_matrix=lgb.Dataset(val_x,label=val_y)
model_B = lgb.train(params_2, train_matrix, num_boost_round=600, valid_sets=valid_matrix,
                  early_stopping_rounds=100
                  )

[1]	valid_0's l2: 76.1725
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l2: 75.7803
[3]	valid_0's l2: 75.4176
[4]	valid_0's l2: 75.2397
[5]	valid_0's l2: 75.0446
[6]	valid_0's l2: 74.9099
[7]	valid_0's l2: 74.7325
[8]	valid_0's l2: 74.5664
[9]	valid_0's l2: 74.4652
[10]	valid_0's l2: 74.375
[11]	valid_0's l2: 74.2574
[12]	valid_0's l2: 74.1673
[13]	valid_0's l2: 74.0576
[14]	valid_0's l2: 73.9683
[15]	valid_0's l2: 73.9259
[16]	valid_0's l2: 73.8728
[17]	valid_0's l2: 73.8265
[18]	valid_0's l2: 73.7466
[19]	valid_0's l2: 73.7068
[20]	valid_0's l2: 73.7162
[21]	valid_0's l2: 73.6715
[22]	valid_0's l2: 73.6617
[23]	valid_0's l2: 73.6381
[24]	valid_0's l2: 73.5786
[25]	valid_0's l2: 73.5617
[26]	valid_0's l2: 73.5588
[27]	valid_0's l2: 73.5471
[28]	valid_0's l2: 73.5423
[29]	valid_0's l2: 73.5317
[30]	valid_0's l2: 73.5263
[31]	valid_0's l2: 73.5237
[32]	valid_0's l2: 73.5148
[33]	valid_0's l2: 73.5145
[34]	valid_0's l2: 73.5193
[35]	valid_0's l2: 73.5236
[3

In [28]:
test_y = user_basic_info[['user_id']]
test_y["gap"] = model_B.predict(test_feature1[feature_list], num_iteration=model.best_iteration).reshape((test_feature1.shape[0], 1))
test_y['pred_date'] = test_y['gap'].map(lambda day: datetime(2017, 9, 1)+timedelta(days=int(day+0.49-1)))

In [31]:
test_cv = test_cv.sort_values("prob",ascending=False)
test_cv = test_cv.merge(test_y[['user_id','pred_date']],on=['user_id'],how='left')
test_sub = test_cv[['user_id','pred_date']]
test_sub[:50000].to_csv("sub_jdata.csv",index=None)

In [33]:
feat_imp = pd.Series(model_A.feature_importance(), feature_list).sort_values(ascending=False)
feat_imp

O_1_30101_once_cost_user_id_sum                60
O_1_30101_order_day_user_id_max                39
A_1_30101_action_day_user_id_max               38
O_4_feature_u1                                 38
O_1_30101_order_day_user_id_min                33
O_3_feature_u1                                 30
O_2_feature_u1                                 29
O_2_30101_once_cost_user_id_sum                27
O_2_30101_o_date_user_id_nunique               27
O_1_gap_end                                    26
user_lv_cd                                     26
A_1_gap_end                                    24
O_3_30101_once_cost_user_id_sum                20
A_1_type1_30101a_num_user_id_sum               19
O_1_30101_comment_day_user_id_min              18
O_1_101_once_cost_user_id_sum                  17
A_2_type1_101a_num_user_id_sum                 17
O_2_30101_comment_day_user_id_max              16
O_1_30_once_cost_user_id_sum                   16
O_1_30101_comment_day_user_id_max              16


In [37]:
#S1 score
import math
def label_change_back(data):
    if(data>=1):
        return 1
    else:
        return 0
val_feature_label1['label_1'] = val_feature_label1.label_1.apply(label_change_back)
val_pred = pd.concat([user_basic_info[['user_id']],val_feature_label1[['label_1']]],axis=1)
val_pred['pred'] = model_A.predict(val_feature_label1[feature_list]).reshape((val_pred.shape[0], 1))
val_pred = val_pred.sort_values("pred",ascending=False).reset_index()
val_pred['index_']=val_pred.index+1
def get_w(data):
    return (1.0/(1.0+math.log(data)))
val_pred['s1_w'] = val_pred.index_.apply(get_w)
val_pred['s1_fenzi'] = val_pred.label_1*val_pred.s1_w
(val_pred[['s1_fenzi']][:50000].sum().values/val_pred[['s1_w']][:50000].sum().values)[0]

0.5077544542988471